## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import load_partition,save_partition, create_train_valid_loaders, RandomTransform

from purrfect.training import train_model
import torch.optim as optim

from purrfect.active_learning import create_new_partition, test_model

from sklearn.model_selection import train_test_split
#from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32
USE_AUTOCAST = True
EARLY_STOPPING_PATIENCE = 5
EARLY_STOPPING_GRACE_PERIOD = 8

## Definición modelo

In [4]:
class GradientMagnitude(nn.Module):
    def __init__(self):
        super(GradientMagnitude, self).__init__()
        # Define Sobel filters for computing gradients in x and y directions
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32,device=DEVICE)
        sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32,device=DEVICE)
        
        # Sobel filters need to be reshaped for convolution: (out_channels, in_channels, height, width)
        # Apply the filters across all channels by expanding them to shape (C, 1, 3, 3)
        self.sobel_x = sobel_x.view(1, 1, 3, 3)
        self.sobel_y = sobel_y.view(1, 1, 3, 3)
    
    def forward(self, x):
        B, C, W, H = x.shape
        
        # Apply Sobel filters to compute gradients in x and y directions for all channels
        grad_x = F.conv2d(x, self.sobel_x.expand(C, 1, 3, 3), groups=C, padding=1)
        grad_y = F.conv2d(x, self.sobel_y.expand(C, 1, 3, 3), groups=C, padding=1)
        
        # Compute gradient magnitude: sqrt(grad_x^2 + grad_y^2)
        grad_mag = torch.sqrt(grad_x**2 + grad_y**2)
        
        return grad_mag
def aggregate_similarity_cosine(tensor):
    # tensor is of shape (B, 2, H, W)
    B, C, H, W = tensor.shape
    #assert C == 2, "The channel dimension must represent 2D vectors (x, y components)"

    # Define shifts for neighboring vectors (up, down, left, right) and stack them
    shift_up = F.pad(tensor[:, :, 1:, :], (0, 0, 0, 1))  # Shift up
    shift_down = F.pad(tensor[:, :, :-1, :], (0, 0, 1, 0))  # Shift down
    shift_left = F.pad(tensor[:, :, :, 1:], (0, 1, 0, 0))  # Shift left
    shift_right = F.pad(tensor[:, :, :, :-1], (1, 0, 0, 0))  # Shift right

    # Stack shifted neighbors into a single tensor of shape (B, 2, H, W, 4)
    neighbors = torch.stack((shift_up, shift_down, shift_left, shift_right), dim=4)  # (B, 2, H, W, 4)
    # Calculate dot product for all neighbors
    dot_product = (tensor.unsqueeze(4) * neighbors).sum(dim=1,keepdim=True)  # (B, 1, H, W, 4)
    # Calculate mean, max, and min along the neighbor dimension (dim=4)
    mean_similarity = dot_product.mean(dim=4, keepdim=True)  # (B, 1, H, W, 1)
    max_similarity = dot_product.max(dim=4, keepdim=True).values  # (B, 1, H, W, 1)
    min_similarity = dot_product.min(dim=4, keepdim=True).values  # (B, 1, H, W, 1)

    # Concatenate mean, max, and min similarities along the channel dimension
    aggregate_similarity = torch.cat((mean_similarity, max_similarity, min_similarity), dim=1)  # (B, 3, H, W,1)

    return aggregate_similarity.squeeze(-1)  # (B, 3, H, W)
class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        self.grad_magnitude = GradientMagnitude()

    def forward(self, x):
        
        # Extract the first two channels (e1 and e2) and the third channel (delta) directly
        e = x[:, :2, :, :]  # Shape (B, 2, H, W)
        delta = x[:, 2:3, :, :]  # Shape (B, 1, H, W)

        # Calculate the magnitude in one step (B, 1, H, W)
        magnitude = torch.norm(e, dim=1, keepdim=True)  # Efficient norm calculation

        # Calculate the angle using atan2 to avoid division by zero and handle quadrant
        angle = 0.5 * torch.atan2(e[:, 0, :, :], e[:, 1, :, :]).unsqueeze(1)  # (B, 1, H, W)

        # Compute the weighted components e1_weighted and e2_weighted
        e_norm = e / magnitude  # Split along channel dimension

        # Compute the cross product
        e = x[:, :2, :, :]  # Shape (B, 2, H, W)
        cross_product = e[:, 0:1, :, :] * e[:, 1:2, :, :]  # Efficient cross-product (B, 1, H, W)

        simmilarity = aggregate_similarity_cosine(e_norm)
        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            e,
            magnitude,
            angle,
            e_norm,
            cross_product,
            delta,
            simmilarity #min max and mean simmilarity
        ], dim=1)
        output = torch.cat([output, self.grad_magnitude(output)], dim=1)

        return output

In [5]:
BN_MOMENTUM = 0.1
def kaiming_init(module,
                 a=0,
                 mode='fan_out',
                 nonlinearity='relu',
                 bias=0,
                 distribution='normal'):
    assert distribution in ['uniform', 'normal']
    if distribution == 'uniform':
        nn.init.kaiming_uniform_(
            module.weight, a=a, mode=mode, nonlinearity=nonlinearity)
    else:
        nn.init.kaiming_normal_(
            module.weight, a=a, mode=mode, nonlinearity=nonlinearity)
    if hasattr(module, 'bias') and module.bias is not None:
        nn.init.constant_(module.bias, bias)
class PA(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=1, stride=1):
        super(PA, self).__init__()

        self.inplanes = inplanes
        self.inter_planes = planes // 2
        self.planes = planes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = (kernel_size - 1) // 2
        ratio = 4

        self.conv_q_right = nn.Conv2d(self.inplanes, 1, kernel_size=1, stride=stride, padding=0, bias=False)
        self.conv_v_right = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                      bias=False)
        # self.conv_up = nn.Conv2d(self.inter_planes, self.planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv_up = nn.Sequential(
            nn.Conv2d(self.inter_planes, self.inter_planes // ratio, kernel_size=1),
            nn.LayerNorm([self.inter_planes // ratio, 1, 1]),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.inter_planes // ratio, self.planes, kernel_size=1)
        )
        self.softmax_right = nn.Softmax(dim=2)
        self.sigmoid = nn.Sigmoid()

        self.conv_q_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # g
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_v_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # theta
        self.softmax_left = nn.Softmax(dim=2)

        self.reset_parameters()
    def reset_parameters(self):
        kaiming_init(self.conv_q_right, mode='fan_in')
        kaiming_init(self.conv_v_right, mode='fan_in')
        kaiming_init(self.conv_q_left, mode='fan_in')
        kaiming_init(self.conv_v_left, mode='fan_in')

        self.conv_q_right.inited = True
        self.conv_v_right.inited = True
        self.conv_q_left.inited = True
        self.conv_v_left.inited = True

    def spatial_pool_mask(self, x):
        input_x = self.conv_v_right(x)

        batch, channel, height, width = input_x.size()

        # [N, IC, H*W]
        input_x = input_x.view(batch, channel, height * width)

        # [N, 1, H, W]
        context_mask = self.conv_q_right(x)

        # [N, 1, H*W]
        context_mask = context_mask.view(batch, 1, height * width)

        # [N, 1, H*W]
        context_mask = self.softmax_right(context_mask)

        # [N, IC, 1]
        # context = torch.einsum('ndw,new->nde', input_x, context_mask)
        context = torch.matmul(input_x, context_mask.transpose(1, 2))

        # [N, IC, 1, 1]
        context = context.unsqueeze(-1)

        # [N, OC, 1, 1]
        context = self.conv_up(context)

        # [N, OC, 1, 1]
        mask_ch = self.sigmoid(context)
        return mask_ch

    def channel_pool_mask(self, x):
        # [N, IC, H, W]
        g_x = self.conv_q_left(x)

        batch, channel, height, width = g_x.size()

        # [N, IC, 1, 1]
        avg_x = self.avg_pool(g_x)

        batch, channel, avg_x_h, avg_x_w = avg_x.size()

        # [N, 1, IC]
        avg_x = avg_x.view(batch, channel, avg_x_h * avg_x_w).permute(0, 2, 1)

        # [N, IC, H*W]
        theta_x = self.conv_v_left(x).view(batch, self.inter_planes, height * width)

        # [N, IC, H*W]
        theta_x = self.softmax_left(theta_x)

        # [N, 1, H*W]
        # context = torch.einsum('nde,new->ndw', avg_x, theta_x)
        context = torch.matmul(avg_x, theta_x)

        # [N, 1, H, W]
        context = context.view(batch, 1, height, width)

        # [N, 1, H, W]
        mask_sp = self.sigmoid(context)
        return mask_sp
    def forward(self, x, y):
        sp_mask = self.spatial_pool_mask(y)
        ch_mask = self.channel_pool_mask(y)
        return x * sp_mask * ch_mask
    

class PSA(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=1, stride=1):
        super(PSA, self).__init__()
        self.pa = PA(inplanes, planes, kernel_size, stride)

    def forward(self, x):
        return self.pa(x,x)

class ConBnRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,bias=True):
        super(ConBnRelu, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                              dilation=dilation, bias=bias)
        self.bn = nn.BatchNorm2d(out_channels, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out
class DoubleConv(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, use_attention=True):
        super(DoubleConv, self).__init__()
        if use_attention:
            self.seq = nn.Sequential(
                ConBnRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
                PSA(out_channels, out_channels),
                ConBnRelu(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True)
            )
        else:
            self.seq = nn.Sequential(
                ConBnRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
                ConBnRelu(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True)
            )

        self.rescale = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=True)

    def forward(self, x):
        return self.seq(x) + self.rescale(x)


# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, i_ch=16):
        """
        :param in_channels: int, number of input channels
        :param out_channels: int, number of output channels
        :param i_ch: int, internal channels to use for the first DoubleConv
        """
        super(UNet, self).__init__()
        # Channel Adder
        self.embedder = nn.Sequential(
            DoubleConv(in_channels, i_ch),  # in_channelsx128x128 -> i_chx128x128
            DoubleConv(i_ch, i_ch),  # i_chx128x128 -> i_chx128x128
        )
        # Encoder
        self.encoder1 = nn.Sequential(
            nn.AvgPool2d(2),  # i_chx128x128 -> i_chx64x64
            DoubleConv(i_ch, i_ch * 2)  # i_chx64x64 -> (i_ch * 2)x64x64
        )
        self.encoder2 = nn.Sequential(
            nn.AvgPool2d(2),  # (i_ch * 2)x64x64 -> (i_ch * 2)x32x32
            DoubleConv(i_ch * 2, i_ch * 4)  # (i_ch * 2)x32x32 -> (i_ch * 4)x32x32
        )
        self.encoder3 = nn.Sequential(
            nn.AvgPool2d(2),  # (i_ch * 4)x32x32 -> (i_ch * 4)x16x16
            DoubleConv(i_ch * 4, i_ch * 8)  # (i_ch * 4)x16x16 -> (i_ch * 8)x16x16
        )
        self.encoder4 = nn.Sequential(
            nn.AvgPool2d(2),  # (i_ch * 8)x16x16 -> (i_ch * 8)x8x8
            DoubleConv(i_ch * 8, i_ch * 16)  # (i_ch * 8)x8x8 -> (i_ch * 16)x8x8
        )

         # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(i_ch * 16, i_ch * 16, kernel_size=8),  # (i_ch * 16)x8x8 -> (i_ch * 16)x1x1
            nn.Flatten(),
            nn.Linear(i_ch * 16, i_ch * 16),  # (i_ch * 16) -> (i_ch * 16)
            nn.ReLU(inplace=True),
            nn.Linear(i_ch * 16, i_ch * 16),  # (i_ch * 16) -> (i_ch * 16)
            nn.ReLU(inplace=True),
            nn.Unflatten(1, (i_ch, 4, 4)),  # Reshape back to (i_ch)x4x4
            nn.ConvTranspose2d(i_ch, i_ch * 16, kernel_size=2, stride=2), # (i_ch)x4x4 -> (i_ch * 16)x8x8
            nn.ReLU(inplace=True)
        )

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(i_ch * 16, i_ch * 8, kernel_size=2, stride=2)  # (i_ch * 16)x8x8 -> (i_ch * 8)x16x16
        self.attention4 = PA(i_ch * 8, i_ch * 8)
        self.decoder4 = DoubleConv(i_ch * 16, i_ch * 8)  # (i_ch * 16)x16x16 -> (i_ch * 8)x16x16

        self.upconv3 = nn.ConvTranspose2d(i_ch * 8, i_ch * 4, kernel_size=2, stride=2)  # (i_ch * 8)x16x16 -> (i_ch * 4)x32x32
        self.attention3 = PA(i_ch * 4, i_ch * 4)
        self.decoder3 = DoubleConv(i_ch * 8, i_ch * 4)  # (i_ch * 8)x32x32 -> (i_ch * 4)x32x32

        self.upconv2 = nn.ConvTranspose2d(i_ch * 4, i_ch * 2, kernel_size=2, stride=2)  # (i_ch * 4)x32x32 -> (i_ch * 2)x64x64
        self.attention2 = PA(i_ch * 2, i_ch * 2)
        self.decoder2 = DoubleConv(i_ch * 4, i_ch * 2)  # (i_ch * 4)x64x64 -> (i_ch * 2)x64x64

        self.upconv1 = nn.ConvTranspose2d(i_ch * 2, i_ch, kernel_size=2, stride=2)  # (i_ch * 2)x64x64 -> i_chx128x128
        self.attention1 = PA(i_ch, i_ch)
        self.decoder1 = DoubleConv(i_ch * 2, i_ch)  # (i_ch * 2)x128x128 -> i_chx128x128

        self.output = nn.Conv2d(i_ch, out_channels, kernel_size=1)  # i_chx128x128 -> out_channelsx128x128

    def forward(self, x):
        x = self.embedder(x)
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        # Bottleneck
        b = self.bottleneck(e4)
        # Decoder with skip connections
        d4 = self.upconv4(b)
        d4 = torch.cat((self.attention4(e3, d4), d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((self.attention3(e2, d3), d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((self.attention2(e1, d2), d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((self.attention1(x, d1), d1), dim=1)
        d1 = self.decoder1(d1)

        out = self.output(d1)
        return out

    def set_trainable_mode(self, mode):
        """
        Freezes or unfreezes weights based on the mode.
        
        :param mode: str, one of ['full', 'encoder', 'decoder']
        """
        if mode == 'full':
            # Unfreeze all layers
            for param in self.parameters():
                param.requires_grad = True
        elif mode == 'encoder':
            # Freeze decoder and output, unfreeze encoder
            self._set_requires_grad(self.embedder, True)
            self._set_requires_grad(self.encoder1, True)
            self._set_requires_grad(self.encoder2, True)
            self._set_requires_grad(self.encoder3, True)
            self._set_requires_grad(self.encoder4, True)
            self._set_requires_grad(self.bottleneck, True)
            
            self._set_requires_grad(self.upconv4, False)
            self._set_requires_grad(self.attention4, False)
            self._set_requires_grad(self.decoder4, False)
            self._set_requires_grad(self.upconv3, False)
            self._set_requires_grad(self.attention3, False)
            self._set_requires_grad(self.decoder3, False)
            self._set_requires_grad(self.upconv2, False)
            self._set_requires_grad(self.attention2, False)
            self._set_requires_grad(self.decoder2, False)
            self._set_requires_grad(self.upconv1, False)
            self._set_requires_grad(self.attention1, False)
            self._set_requires_grad(self.decoder1, False)
            self._set_requires_grad(self.output, False)
        elif mode == 'decoder':
            # Freeze encoder, unfreeze decoder
            self._set_requires_grad(self.embedder, False)
            self._set_requires_grad(self.encoder1, False)
            self._set_requires_grad(self.encoder2, False)
            self._set_requires_grad(self.encoder3, False)
            self._set_requires_grad(self.encoder4, False)
            self._set_requires_grad(self.bottleneck, False)

            self._set_requires_grad(self.upconv4, True)
            self._set_requires_grad(self.attention4, True)
            self._set_requires_grad(self.decoder4, True)
            self._set_requires_grad(self.upconv3, True)
            self._set_requires_grad(self.attention3, True)
            self._set_requires_grad(self.decoder3, True)
            self._set_requires_grad(self.upconv2, True)
            self._set_requires_grad(self.attention2, True)
            self._set_requires_grad(self.decoder2, True)
            self._set_requires_grad(self.upconv1, True)
            self._set_requires_grad(self.attention1, True)
            self._set_requires_grad(self.decoder1, True)
            self._set_requires_grad(self.output, True)

    def _set_requires_grad(self, module, requires_grad):
        """
        Sets the requires_grad attribute for all parameters in a module.

        :param module: nn.Module
        :param requires_grad: bool
        """
        for param in module.parameters():
            param.requires_grad = requires_grad

class Magnitude(nn.Module):
    def __init__(self, in_channels):
        super(Magnitude, self).__init__()
        self.conv = nn.Sequential(
            DoubleConv(in_channels, 16, use_attention=False), # -> 16x128x128
            nn.AvgPool2d(4), # -> 16x32x32
            DoubleConv(16, 8, use_attention=False), # -> 8x32x32
            nn.AvgPool2d(4), # -> 8x8x8
            nn.Conv2d(8, 1, kernel_size=8) # -> 1x1x1
        )
    def forward(self, x):
        return  self.conv(x)
class KappaPredictor(nn.Module):
    def __init__(self):
        super(KappaPredictor, self).__init__()
        total_channels = 22
        self.channel_adder = ChannelAdder()
        self.bn = nn.BatchNorm2d(total_channels)
        self.unet = UNet(total_channels, 1, i_ch=32)
    def set_trainable_mode(self, mode):
        # Iterate over the sequential module's layers
        self.unet.set_trainable_mode(mode)
        for param in self.bn.parameters():
            param.requires_grad = mode != 'decoder'  # Unfreeze in 'full' and 'encoder' mode
        
    def forward(self, x):
        x = self.channel_adder(x)
        return self.unet(x)
        
        

## Creación particion inicial

In [6]:
def create_next_partitions(current_partition,k=1):#Creacion de particiones train y valid
    init_partition = []
    for i in range(k):
        init_partition += load_partition(f"partition_{k*current_partition+(i+1)}.json")
        #print(f"partition_{k*current_partition+(i+1)}.json")

    train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
    save_partition(f"partition_{current_partition+1}_train.json","partitions",train_partition)
    save_partition(f"partition_{current_partition+1}_val.json","partitions",val_partition)

In [7]:
def load_best_model_so_far(model, last_saved_partition):
    best_loss = float('inf')
    best_partition = 0
    for partition in range(1,last_saved_partition+1):
        checkpoint = torch.load(f"models/last_checkpoint_partition_{partition}.pth",weights_only=False)
        if checkpoint['best_loss'] < best_loss:
            best_loss = checkpoint['best_loss']
            best_partition = partition
    model.load_state_dict(torch.load(f"models/best_model_partition_{best_partition}.pth",weights_only=True))
    print(f"Loaded best model from partition {best_partition} with loss {best_loss}")
def load_last_best_model(model, last_saved_partition):
    #for from last_saved_partition to 1 looking if exists a best model file. if there is then load it and return
    for partition in range(last_saved_partition,0,-1):
        if os.path.exists(f"models/best_model_partition_{partition}.pth"):
            model.load_state_dict(torch.load(f"models/best_model_partition_{partition}.pth",weights_only=True))
            print(f"Loaded best model from partition {partition}")
            return
    print(f"There is no best model saved")

In [8]:
create_next_partitions(0,1)

In [9]:
def many_partitions_v2(start,end,model,criterion,transform=None,full_frecuency=2):
    for current_partition in range(start,end):
        if current_partition %full_frecuency==1:
            print(f"Partition {current_partition}: training full")
            model.set_trainable_mode('full')
        else:
            print(f"Partition {current_partition}: training decoder")
            model.set_trainable_mode('decoder')
        load_last_best_model(model,current_partition-1)
        create_next_partitions(current_partition-1)
        train_loader, val_loader = create_train_valid_loaders(
            f"partition_{current_partition}_train.json",
            f"partition_{current_partition}_val.json",
            "partitions",
            batch_size=BATCH_SIZE,
            transform=transform,
        )
        best_model_path = os.path.join(
            "models", f"best_model_partition_{current_partition}.pth"
        )
        last_checkpoint_path = os.path.join(
            "models", f"last_checkpoint_partition_{current_partition}.pth"
        )
        optimizer = optim.AdamW(model.parameters())
        train_model(
            model,
            train_loader,
            val_loader,
            best_model_path,
            last_checkpoint_path,
            criterion,
            optimizer,
            num_epochs=50,
            device=DEVICE,
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            use_autocast=USE_AUTOCAST,
            early_stopping_grace_period=EARLY_STOPPING_GRACE_PERIOD,
        )
        if os.path.exists(best_model_path):
            model.load_state_dict(torch.load(best_model_path,weights_only=True))
            test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE,experiment_name=f"adamw_atunet_freeze__{current_partition}")
        else:
            print("No best model found in partition",current_partition)
        

## Carga modelo

In [10]:
#Define model
model = KappaPredictor().to(DEVICE)
# Define Loss
criterion = torch.nn.L1Loss()
transform = RandomTransform()

In [11]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

26400053

In [18]:
many_partitions_v2(1,10,model,criterion,transform=transform)

Partition 1: training full
There is no best model saved
Epoch [1/50]


Validate Epoch 1: 100%|██████████| 39/39 [00:07<00:00,  5.52it/s, WMAPE=7.65, DICE=0.432, DPEAKS=201, Loss=0.0402]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 39/39 [00:06<00:00,  6.11it/s, WMAPE=9.37, DICE=0.599, DPEAKS=190, Loss=0.0501]


Epoch [3/50]


...

Validate Epoch 16: 100%|██████████| 39/39 [00:06<00:00,  6.20it/s, WMAPE=2.17, DICE=0.0723, DPEAKS=39.7, Loss=0.00643]


Epoch [17/50]


Validate Epoch 17: 100%|██████████| 39/39 [00:06<00:00,  6.17it/s, WMAPE=2.05, DICE=0.0743, DPEAKS=46.5, Loss=0.00643]


early stopping: 5 epochs without improvement
Training complete.


Validate Epoch test: 100%|██████████| 971/971 [03:27<00:00,  4.68it/s, WMAPE=2.07, DICE=0.0726, DPEAKS=39.9, Loss=0.00632]


In [19]:
many_partitions_v2(10,30,model,criterion,transform=transform)

Partition 10: training decoder
Loaded best model from partition 9
Epoch [1/50]


Validate Epoch 1: 100%|██████████| 39/39 [00:07<00:00,  5.14it/s, WMAPE=1.95, DICE=0.0711, DPEAKS=38.8, Loss=0.00626]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 39/39 [00:06<00:00,  6.09it/s, WMAPE=2.04, DICE=0.0709, DPEAKS=38.9, Loss=0.00648]


Epoch [3/50]


...

Epoch [4/50]


Validate Epoch 4: 100%|██████████| 39/39 [00:06<00:00,  5.64it/s, WMAPE=2.55, DICE=0.059, DPEAKS=33.1, Loss=0.00742]


Saving best model
Epoch [5/50]


Train Epoch 5:  21%|██        | 32/154 [00:14<00:54,  2.24it/s, loss=0.00658]

KeyboardInterrupt: 

In [12]:
many_partitions_v2(21,31,model,criterion,transform=transform,full_frecuency=99)

Partition 21: training decoder
Loaded best model from partition 20
Epoch [1/50]


Validate Epoch 1: 100%|██████████| 39/39 [00:06<00:00,  5.61it/s, WMAPE=1.84, DICE=0.0571, DPEAKS=32.3, Loss=0.00553]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 39/39 [00:06<00:00,  5.63it/s, WMAPE=1.75, DICE=0.0565, DPEAKS=30, Loss=0.00529]


Saving best model
Epoch [3/50]


...

Validate Epoch 7: 100%|██████████| 39/39 [00:06<00:00,  5.58it/s, WMAPE=1.66, DICE=0.0571, DPEAKS=36.3, Loss=0.00507]


Epoch [8/50]


Validate Epoch 8: 100%|██████████| 39/39 [00:06<00:00,  5.59it/s, WMAPE=1.87, DICE=0.0563, DPEAKS=30.2, Loss=0.0055]


early stopping: 7 epochs without improvement
Training complete.


Validate Epoch test: 100%|██████████| 971/971 [03:42<00:00,  4.36it/s, WMAPE=1.65, DICE=0.0564, DPEAKS=30.4, Loss=0.00509]


In [13]:
many_partitions_v2(31,51,model,criterion,transform=transform,full_frecuency=99)

Partition 31: training decoder
Loaded best model from partition 30
Epoch [1/50]


Validate Epoch 1: 100%|██████████| 39/39 [00:08<00:00,  4.80it/s, WMAPE=1.72, DICE=0.0569, DPEAKS=27.7, Loss=0.0052]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 39/39 [00:06<00:00,  5.69it/s, WMAPE=1.64, DICE=0.0563, DPEAKS=30.2, Loss=0.00495]


Saving best model
Epoch [3/50]


...

Validate Epoch 10: 100%|██████████| 39/39 [00:06<00:00,  6.10it/s, WMAPE=1.69, DICE=0.0555, DPEAKS=30.3, Loss=0.00503]


Epoch [11/50]


Validate Epoch 11: 100%|██████████| 39/39 [00:06<00:00,  6.04it/s, WMAPE=1.72, DICE=0.0559, DPEAKS=25.9, Loss=0.00506]


early stopping: 5 epochs without improvement
Training complete.


Validate Epoch test: 100%|██████████| 971/971 [03:29<00:00,  4.64it/s, WMAPE=1.63, DICE=0.0556, DPEAKS=29.2, Loss=0.00487]


In [14]:
many_partitions_v2(51,71,model,criterion,transform=transform,full_frecuency=99)

Partition 51: training decoder
Loaded best model from partition 50
Epoch [1/50]


Validate Epoch 1: 100%|██████████| 39/39 [00:07<00:00,  5.06it/s, WMAPE=1.65, DICE=0.0552, DPEAKS=28.5, Loss=0.005]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 39/39 [00:06<00:00,  6.05it/s, WMAPE=1.69, DICE=0.0557, DPEAKS=28.1, Loss=0.00527]


Epoch [3/50]


...

Validate Epoch 13: 100%|██████████| 39/39 [00:06<00:00,  5.69it/s, WMAPE=1.67, DICE=0.0551, DPEAKS=27.7, Loss=0.00499]


Epoch [14/50]


Validate Epoch 14: 100%|██████████| 39/39 [00:06<00:00,  5.70it/s, WMAPE=1.89, DICE=0.0567, DPEAKS=29.9, Loss=0.00585]


early stopping: 5 epochs without improvement
Training complete.


Validate Epoch test: 100%|██████████| 971/971 [03:41<00:00,  4.38it/s, WMAPE=1.6, DICE=0.0551, DPEAKS=28.6, Loss=0.00481]


In [15]:
many_partitions_v2(71,91,model,criterion,transform=transform,full_frecuency=99)

Partition 71: training decoder
Loaded best model from partition 70
Epoch [1/50]


Validate Epoch 1: 100%|██████████| 39/39 [00:08<00:00,  4.84it/s, WMAPE=1.76, DICE=0.0561, DPEAKS=29.4, Loss=0.00555]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 39/39 [00:06<00:00,  5.76it/s, WMAPE=1.72, DICE=0.0552, DPEAKS=28.2, Loss=0.00509]


Saving best model
Epoch [3/50]


...

early stopping: 5 epochs without improvement
Training complete.


Validate Epoch test: 100%|██████████| 971/971 [03:28<00:00,  4.65it/s, WMAPE=1.61, DICE=0.055, DPEAKS=28.2, Loss=0.00486]


Partition 72: training decoder
Loaded best model from partition 71
Epoch [1/50]


Train Epoch 1:   9%|▉         | 14/154 [00:04<00:49,  2.82it/s, loss=0.00647]

KeyboardInterrupt: 